In [1]:
import pandas as pd
import os

env = os.environ.get('Environment')
print(f"Environment: {env!r}")

Environment: 'prod'


In [2]:
%reload_ext jupyter_dmdg

## Non-compliant tags with SD-000478

In [3]:
# non-compliant tags at L3

In [4]:
%%athena_to_df --out L3
SELECT DISTINCT 
    tag_id,
    site_id,
    building_id,
    source_name, 
    SUBSTRING(premise_id,1,3) AS premise    
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    SUBSTRING(premise_id,1,3) NOT IN (
        SELECT 
            trigram 
        FROM "prod-plantco-referential-data"."prod_plantco_referential_hierarchy"
        WHERE level = '3'
    )
    AND SUBSTRING(premise_id,1,3) != ''
    AND source_name != ''

In [5]:
# non-compliant tags at L4

In [6]:
%%athena_to_df --out L4
SELECT DISTINCT 
    tag_id, 
    site_id,
    building_id,
    source_name, 
    SUBSTRING(main_function_id,1,3) AS main_function
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    SUBSTRING(main_function_id,1,3) NOT IN (
        SELECT 
            trigram 
        FROM "prod-plantco-referential-data"."prod_plantco_referential_hierarchy"
        WHERE level = '4'
    )
    AND SUBSTRING(main_function_id,1,3) != ''
    AND source_name != ''

In [7]:
# non-compliant tags at L5

In [8]:
%%athena_to_df --out L5_2letters
SELECT DISTINCT 
    tag_id, 
    site_id,
    building_id,
    source_name, 
    SUBSTRING(secondary_function_id,1,2) AS secondary_function
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    SUBSTRING(secondary_function_id,1,2) NOT IN (
        SELECT 
            trigram 
        FROM "prod-plantco-referential-data"."prod_plantco_referential_hierarchy"
        WHERE level = '5'
    )
    AND secondary_function_id != ''
    AND source_name != ''
    AND SUBSTRING(secondary_function_id,3,1) BETWEEN '0' AND '9'

In [9]:
%%athena_to_df --out L5_3letters
SELECT DISTINCT 
    tag_id, 
    site_id,
    building_id,
    source_name, 
    SUBSTRING(secondary_function_id,1,3) AS secondary_function
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    SUBSTRING(secondary_function_id,1,3) NOT IN (
        SELECT 
            trigram 
        FROM "prod-plantco-referential-data"."prod_plantco_referential_hierarchy"
        WHERE level = '5'
    )
    AND secondary_function_id != ''
    AND source_name != ''
    AND SUBSTRING(secondary_function_id,4,1) BETWEEN '0' AND '9'
    AND NOT SUBSTRING(secondary_function_id,3,1) BETWEEN '0' AND '9'

In [10]:
%%athena_to_df --out L5_4letters
SELECT DISTINCT 
    tag_id, 
    site_id,
    building_id,
    source_name, 
    SUBSTRING(secondary_function_id,1,4) AS secondary_function
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    SUBSTRING(secondary_function_id,1,4) NOT IN (
        SELECT 
            trigram 
        FROM "prod-plantco-referential-data"."prod_plantco_referential_hierarchy"
        WHERE level = '5'
    )
    AND secondary_function_id != ''
    AND source_name != ''
    AND SUBSTRING(secondary_function_id,5,1) BETWEEN '0' AND '9'
    AND NOT SUBSTRING(secondary_function_id,4,1) BETWEEN '0' AND '9'
    AND NOT SUBSTRING(secondary_function_id,3,1) BETWEEN '0' AND '9'

In [11]:
L5 = pd.concat([L5_2letters, L5_3letters, L5_4letters], axis=0)

In [12]:
# non-compliant tags at LH

In [13]:
L3_L4 = L3.merge(L4, on=['tag_id', 'site_id', 'source_name', 'building_id'] , how='outer')
LH = L3_L4.merge(L5, on=['tag_id', 'site_id', 'source_name', 'building_id'] , how='outer')

## Non-compliant tags with SD-000139

In [14]:
# non-compliant tags at SYS

In [15]:
%%athena_to_df --out SYS
SELECT DISTINCT 
    tag_id, 
    site_id,
    building_id,
    source_name, 
    system
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    system NOT IN (
        SELECT DISTINCT 
            sys  
        FROM "prod-plantco-referential-data"."prod_plantco_referential_tags"
    )
    AND system != ''
    AND source_name != ''

In [16]:
# non-compliant tags at SUBSYS

In [17]:
%%athena_to_df --out SUBSYS
SELECT DISTINCT 
    tag_id,
    site_id,
    building_id,
    source_name, 
    subsystem
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    subsystem NOT IN (
        SELECT DISTINCT 
            sub 
        FROM "prod-plantco-referential-data"."prod_plantco_referential_tags"
    )
    AND subsystem != ''
    AND source_name != ''

In [18]:
# non-compliant tags at TYP

In [19]:
%%athena_to_df --out TYP
SELECT DISTINCT 
    tag_id, 
    site_id,
    building_id,
    source_name, 
    measure_type
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    measure_type NOT IN (
        SELECT DISTINCT 
            mea  
        FROM "prod-plantco-referential-data"."prod_plantco_referential_tags"
    )
    AND measure_type != ''
    AND source_name != ''

In [20]:
# non-compliant tags at MEA

In [21]:
%%athena_to_df --out MEA
SELECT DISTINCT 
    tag_id, 
    site_id,
    building_id,
    source_name, 
    measure
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    measure NOT IN (
        SELECT DISTINCT 
            typ  
        FROM "prod-plantco-referential-data"."prod_plantco_referential_tags"
    )
    AND measure != ''
    AND source_name != ''

In [22]:
# non-compliant tags at Tagname Part

In [23]:
SYSSUB = pd.merge(SYS, SUBSYS, on=['tag_id', 'site_id', 'source_name', 'building_id'] , how='outer')
SYSSUBTYP = pd.merge(SYSSUB, TYP, on=['tag_id', 'site_id', 'source_name', 'building_id'] , how='outer')
tag_name = pd.merge(SYSSUBTYP, MEA, on=['tag_id', 'site_id', 'source_name', 'building_id'] , how='outer')

## Non-compliant tags

In [24]:
# at this point we still consider the tag_id already changed and we have duplicated tag_ids 

In [25]:
wrong_renamed_tags = pd.merge(LH, tag_name, on=['tag_id', 'site_id', 'source_name', 'building_id'], how='outer')

### 01 Delete from "wrong_renamed_tags" all combinations of site_id, building_id and source_name that appear more times in "all_t" than in "wrong_renamed_tags"

In [26]:
# query all tags

In [27]:
%%athena_to_df --out all_t
SELECT DISTINCT 
    tag_id AS tag_t,
    site_id,
    building_id,
    source_name
FROM "prod_plant_connectivity_prepared_data"."tag_metadata"
WHERE 
    premise_id != ''
    AND main_function_id != ''
    AND secondary_function_id != ''
    AND system != ''
    AND subsystem != ''
    AND measure_type != ''
    AND measure != ''
    AND source_name != ''

In [28]:
# from now on we don't include any combination that presents a compliant tag_id

In [29]:
wrong_renamed_tags_cols = wrong_renamed_tags[['tag_id', 'building_id', 'site_id', 'source_name']]
wrong_renamed_tags_counts = wrong_renamed_tags_cols.groupby(['site_id', 'building_id', 'source_name']).nunique()
all_t_counts = all_t.groupby(['site_id', 'building_id', 'source_name']).nunique()
comparison_df = pd.merge(wrong_renamed_tags_counts, all_t_counts, on=['site_id', 'building_id', 'source_name'], how='left')
compliant_combinations = comparison_df[comparison_df['tag_id'] < comparison_df['tag_t']]
compliant_combinations = compliant_combinations.drop('tag_id', axis=1)
merge2drop = pd.merge(wrong_renamed_tags, compliant_combinations, on=['site_id', 'building_id', 'source_name'], how='left')
truly_wrong_renamed_tags = merge2drop[merge2drop['tag_t'].isnull()]
truly_wrong_renamed_tags = truly_wrong_renamed_tags.drop('tag_t', axis=1)

### 02 Keep only the most recent non-compliant tag_id per each combination

In [30]:
# query all distinct tag_ids in the timeseries table, keeping always the most recent one

In [31]:
%%athena_to_df --out distinct_tag_id
SELECT 
    tag_id, 
    MAX(ts) AS recent_date
FROM "prod_plant_connectivity_prepared_data"."tag_timeserie"
GROUP BY tag_id 

In [32]:
# from now on we don't include any combination that presents a compliant tag_id and we keep only the most recent non-compliant tag_id per each non-compliant combination

In [33]:
truly_wrong_renamed_tags_with_ts = pd.merge(truly_wrong_renamed_tags, distinct_tag_id, on=['tag_id'], how='left')
truly_wrong_renamed_tags_with_ts = truly_wrong_renamed_tags_with_ts.sort_values(by='recent_date', ascending=False)
current_non_compliant_tags = truly_wrong_renamed_tags_with_ts.drop_duplicates(subset=['site_id', 'building_id', 'source_name'], keep='first')

### 03 Drop duplicated tag_ids

In [34]:
non_compliant_tags = current_non_compliant_tags.drop_duplicates(subset=['tag_id'], keep='first')

### 04 Due to there are trigrams that do not appear in the reference table we have in the Data Lake

In [35]:
# remove tagname trigrams that appear in SD-000139 but are not in Data Lake

In [36]:
non_compliant_tags = non_compliant_tags.copy()
df_139 = pd.read_csv('data/SD-000139.csv')
import numpy as np
non_compliant_tags['system'] = np.where(non_compliant_tags['system'].isin(df_139['SYS']), '', non_compliant_tags['system'])
non_compliant_tags['subsystem'] = np.where(non_compliant_tags['subsystem'].isin(df_139['SUB']), '', non_compliant_tags['subsystem'])
non_compliant_tags['measure_type'] = np.where(non_compliant_tags['measure_type'].isin(df_139['MEA']), '', non_compliant_tags['measure_type'])
non_compliant_tags['measure'] = np.where(non_compliant_tags['measure'].isin(df_139['TYP']), '', non_compliant_tags['measure'])

In [37]:
# delete tags that are actually properly renamed if we look for trigrams in the standards and not in Data Lake

In [38]:
columns_to_check = ['premise', 'main_function', 'secondary_function', 'system', 'subsystem', 'measure_type', 'measure']
non_compliant_tags[columns_to_check] = non_compliant_tags[columns_to_check].fillna('')
non_compliant_tags = non_compliant_tags[~(non_compliant_tags[columns_to_check].apply(lambda x: x == '', axis=1).all(axis=1))]
non_compliant_tags = non_compliant_tags.drop(['source_name', 'recent_date'], axis=1)
non_compliant_tags = non_compliant_tags.reset_index(drop=True)

## Statistics for tags

In [39]:
# count how many distinct combinations of site_id, building_id and source_name there are per site_id in the metadata table
# if two distinct combinations share the same tag_id, count only one of them

In [40]:
all_t_distinct_combinations = all_t.drop_duplicates(subset=['source_name', 'site_id', 'building_id'])
all_t_distinct_combinations = all_t_distinct_combinations.drop_duplicates(subset=['tag_t'])
count_all_t_distinct_combinations = all_t_distinct_combinations.groupby('site_id').size().reset_index(name='nb_tags')

In [41]:
# count how many non-compliant tags per site there are

In [42]:
count_non_compliant_tags = non_compliant_tags.groupby('site_id').size().reset_index(name='nb_non_compliant_tags')

In [43]:
# calculate the compliance percentage

In [44]:
for_statistics = count_all_t_distinct_combinations.merge(count_non_compliant_tags, on='site_id' , how='outer')
for_statistics['nb_non_compliant_tags'].fillna(0, inplace=True)
for_statistics['pct_compliance'] = 100-(100*for_statistics['nb_non_compliant_tags']/for_statistics['nb_tags'])
for_statistics['pct_compliance'] = for_statistics['pct_compliance'].round(2)

## Have both list with non-compliant tags and compliance statistics for tags in the dedicated workspace

In [45]:
from src.utils import write_into_table
workspace_prefix = 'public-plant-co-data-quality-20230520193210554'

In [46]:
# list with non-compliant tags in the dedicated workspace

In [47]:
tag_table_name = 'non_compliant_tags'
write_into_table(
    non_compliant_tags,
    tag_table_name.lower(),
    workspace_prefix
)

In [48]:
# compliance statistics for tags in the dedicated workspace

In [49]:
tag_stats_table_name = 'stats4tags'
write_into_table(
    for_statistics,
    tag_stats_table_name.lower(),
    workspace_prefix
)